In [ ]:
pip uninstall numpy

In [ ]:
pip install numpy==1.19.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.5.3 which is incompatible.
lucid 0.3.10 requires numpy<=1.19, but you have numpy 1.19.5 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import numpy
numpy.version.version

'1.19.5'

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [ ]:
import tensorflow as tf
from tensorflow.python.layers.core import Dense
import tensorflow.contrib.keras as keras
from keras.preprocessing.sequence import pad_sequences
import os
import numpy as np
import pandas as pd
from pandas import DataFrame
import time
import random
from PIL import Image
#from trec_eval import trec_eval
import nltk
import copy

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Using TensorFlow backend.


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
config = tf.ConfigProto()

In [ ]:
random.seed(1)
np.random.seed(1)

# Useful Functions

In [ ]:
def negative_samples(num_samples,toplist,downlist,combinationlist):
    sampledata = []
    num = 0
    while num < num_samples:
        top = random.sample(toplist,1)[0]
        down = random.sample(downlist,1)[0]
        if top+down not in combinationlist:
            sampledata.append((top,down,-1))
            num += 1
    return sampledata

In [ ]:
def batch_to_input(batch,imglist,topidlist,downidlist):
    img1 = []#for top
    img2 = []#for down
    img1id = []
    img2id = []
    label = []
    for instance in batch:
        img1.append(imglist[instance[0]])
        img2.append(imglist[instance[1]])
        img1id.append(topidlist[instance[0]])
        img2id.append(downidlist[instance[1]])
        commentid = instance[2]
        if commentid == -1:
            label.append([1,0])
        else:
            label.append([0,1])
    return np.array(img1),np.array(img2),np.array(img1id),np.array(img2id),np.array(label)

In [ ]:
def get_batches(data,batch_size,toplist,downlist,combinationlist,imglist,topidlist,downidlist):
    datacopy = copy.copy(data)
    datacopy.extend(negative_samples(len(datacopy),toplist,downlist,combinationlist))
    random.shuffle(datacopy)
    for batch_i in range(0,len(datacopy)//batch_size+1):
        start_i = batch_i*batch_size
        batch = datacopy[start_i:start_i+batch_size]         
        yield batch_to_input(batch,imglist,topidlist,downidlist)

In [ ]:
def build_evaluation_batch(fixitem,itemlist,state,imglist,topidlist,downidlist):
    img1 = []
    img2 = []
    img1id = []
    img2id = []
    if state == 0:#top,downs
        for item in itemlist:
            img1.append(imglist[fixitem])
            img2.append(imglist[item])
            img1id.append(topidlist[fixitem])
            img2id.append(downidlist[item])
    if state == 1:#down,tops
        for item in itemlist:
            img1.append(imglist[item])
            img2.append(imglist[fixitem])
            img1id.append(topidlist[item])
            img2id.append(downidlist[fixitem])
    return np.array(img1),np.array(img2),np.array(img1id),np.array(img2id)

In [ ]:
def accuracy(label,prediction):
    return (label.argmax(axis=1) == prediction.argmax(axis=1)).sum()/len(label)

In [ ]:
def prepare_evaluation(data_path):  
    with open(data_path,'r') as f:
        content = f.readlines()
    data = {}
    orderlist = []
    labellist = {}
    query_number = 0
    for line in content:
        line = line.split('\t')
        print(line)
        line[2]=line[0:1]
        if data.get(line[0]) != None:
            data[line[0]].append(line[1])
        else:
            data[line[0]] = [line[1]] 
            labellist[query_number] = {}
            query_number += 1
            orderlist.append(line[0])
        if int(line[2]) == 1:
            labellist[query_number-1][line[1]] = 1
        else:
            labellist[query_number-1][line[1]] = 0
    return data,orderlist,labellist

In [ ]:
def trec_evaluation(qrel_file_path,trec_file_path,trec):
    with open(trec_file_path,'w') as f:
        i = 0
        while i < len(trec):
            j = 0 
            while j < len(trec[i]):
                f.write(str(i)+' '+'Q0 '+trec[i][j][0]+' '+str(j+1)+' '+str(trec[i][j][1])+' '+'Exp'+'\n')#注意排序从1开始
                j += 1
            i += 1   
    result = trec_eval(qrel_file_path,trec_file_path)
    print(result)
    return result

In [ ]:
def auc_evaluation(labellist,trec):
    query_number = 0
    record = []
    while query_number < len(trec):
        negative = 0
        temp = []
        for combination in trec[query_number]:
            if labellist[query_number][combination[0]] == 1:
                temp.append(negative)
            else:
                negative += 1
        record.extend([(negative-val)/float(negative) for val in temp])
        query_number += 1
    auc = np.array(record).mean()
    print(auc)
    return auc

# Prepare Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/datasets.zip

Streaming output truncated to the last 5000 lines.
  inflating: datasets/matching/img/77420971.jpg  
  inflating: datasets/matching/img/77424407.jpg  
  inflating: datasets/matching/img/77425043.jpg  
  inflating: datasets/matching/img/77425155.jpg  
  inflating: datasets/matching/img/77429512.jpg  
  inflating: datasets/matching/img/7743223.jpg  
  inflating: datasets/matching/img/77435835.jpg  
  inflating: datasets/matching/img/77441965.jpg  
  inflating: datasets/matching/img/77445037.jpg  
  inflating: datasets/matching/img/77449041.jpg  
  inflating: datasets/matching/img/77452766.jpg  
  inflating: datasets/matching/img/77452771.jpg  
  inflating: datasets/matching/img/77453343.jpg  
  inflating: datasets/matching/img/77454076.jpg  
  inflating: datasets/matching/img/77454970.jpg  
  inflating: datasets/matching/img/77455215.jpg  
  inflating: datasets/matching/img/7747014.jpg  
  inflating: datasets/matching/img/77475534.jpg  
  inflating: datasets/matching/img/77476317.jpg  
 

In [ ]:
toplist = []
topidlist = {}
with open('/content/drive/MyDrive/Mini_project/CG/train/newtoplist.dat','r') as f:#in toplist, the first col is img_name of top, the second col is comments_index
    content = f.readlines()
for line in content:
    line = line[:-1]
    toplist.append(line)
    topidlist[line] = len(topidlist)
toplist

FileNotFoundError: ignored

In [ ]:
toplist[214]= '194796874'

In [ ]:
topidlist['194796874'] = topidlist['19479687']
del topidlist['19479687']

In [ ]:
print(topidlist['194796874'])

214


In [ ]:
downlist = []
downidlist = {}
with open('/content/drive/MyDrive/Mini_project/CG/train/newdownlist.dat','r') as f:#in downlist, the first col is img_name of down(i.e. bottom), the second col is comments_index
    content = f.readlines()
for line in content:
    line = line[:-1]
    downlist.append(line)
    downidlist[line] = len(downidlist)
downlist

['161425425',
 '183639308',
 '149494468',
 '185256499',
 '187619920',
 '192304171',
 '154908928',
 '195127827',
 '201020544',
 '184706162',
 '150894162',
 '181131950',
 '179103801',
 '189834308',
 '119380232',
 '188007967',
 '188977857',
 '182883012',
 '190945208',
 '190044021',
 '195070550',
 '184478646',
 '187329463',
 '164683144',
 '181298080',
 '144475071',
 '195590627',
 '191300349',
 '187264196',
 '179734357',
 '134944806',
 '185315961',
 '154789765',
 '182028192',
 '194791452',
 '95459757',
 '154937089',
 '149682245',
 '192966244',
 '196453490',
 '161361175',
 '123524655',
 '194004637',
 '166367125',
 '114499573',
 '196049545',
 '174451466',
 '166460249',
 '168939642',
 '183486674',
 '189165180',
 '123005289',
 '177931718',
 '167673148',
 '178356096',
 '159216863',
 '187132508',
 '172591117',
 '184955271',
 '167795246',
 '163583573',
 '184133730',
 '146006510',
 '145117145',
 '158699966',
 '194750247',
 '132332610',
 '192426021',
 '187494878',
 '175063741',
 '184723300',
 '18507

In [ ]:
downidlist

{'101347443': 134,
 '103449172': 84,
 '106239071': 104,
 '109344972': 96,
 '113657786': 118,
 '114499573': 44,
 '114600316': 143,
 '116289573': 193,
 '117016854': 130,
 '119101477': 76,
 '119380232': 14,
 '120839784': 214,
 '120960130': 202,
 '123005289': 51,
 '123524655': 41,
 '126770735': 212,
 '132332610': 66,
 '134944806': 30,
 '135628910': 234,
 '136462851': 112,
 '140208236': 152,
 '140686067': 171,
 '140964599': 114,
 '141789192': 238,
 '143550669': 100,
 '144475071': 25,
 '144504823': 211,
 '144816858': 232,
 '145117145': 63,
 '145948674': 200,
 '146006510': 62,
 '146164074': 237,
 '146838171': 197,
 '147409003': 155,
 '148176415': 78,
 '148250134': 94,
 '149136509': 239,
 '149481531': 151,
 '149494468': 2,
 '149607212': 183,
 '149673015': 196,
 '149682245': 37,
 '150894162': 10,
 '151507227': 191,
 '151539200': 81,
 '152126288': 77,
 '152716258': 173,
 '153637817': 138,
 '154718285': 80,
 '154789765': 32,
 '154908928': 6,
 '154937089': 36,
 '155738457': 221,
 '156343228': 162,

In [ ]:
combinationlist = set()
with open('/content/drive/MyDrive/Mini_project/CG/train/newcombinationlist.dat','r') as f:#in combinationlist, the first col is img_name of top, the second col is img_name of down(i.e. bottom), the third col is comments_index    
    content = f.readlines()
for line in content:
    line = line[:-2].split('\t')
    combinationlist.add(line[0]+line[1])
combinationlist

{'11385229919635234',
 '12555251618713250',
 '13601146813562891',
 '13601146814481685',
 '13601146816676619',
 '13601146818202819',
 '13601146819358698',
 '13922304214447507',
 '14868494814096459',
 '15099990615212628',
 '15146070210934497',
 '15467443219604954',
 '15752815416142542',
 '16005791215478976',
 '16005791216136117',
 '16005791216373662',
 '16005791217781935',
 '16005791217835609',
 '16224848216727415',
 '16299169016646024',
 '16430070016287334',
 '16501018217499357',
 '1679022481566088',
 '16790224817921039',
 '16790224817937446',
 '16959197216754145',
 '17169261018852283',
 '17278534011365778',
 '17414117319631539',
 '18060651016893964',
 '18127535418525649',
 '18166655117252862',
 '1820954838355296',
 '18336167117910380',
 '18398897211449957',
 '18432316618288301',
 '18461429317696111',
 '18507064918458720',
 '18574285919537759',
 '18603127019242602',
 '18759352218798052',
 '18794322318732946',
 '19043628614817641',
 '19097794118355537',
 '19097796919101249',
 '1912127661

In [ ]:
imglist = {}
for img_idx in toplist:
    img = Image.open('/content/datasets/CG/img/'+img_idx+'.jpg')
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = np.array(img)
    img = img/255.0
    imglist[img_idx] = img
for img_idx in downlist:
    img = Image.open('/content/datasets/CG/img/'+img_idx+'.jpg')
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = np.array(img)
    img = img/255.0
    imglist[img_idx] = img
imglist

{'100408142': array([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
 
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
 
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
 
        ...,
 
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
 
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
 
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]]), '100428440': array([[[1., 1., 1.],
       

# Build Model

In [ ]:
def get_input():
    img1 = tf.placeholder(tf.float32,[None,150,150,3],'img1')
    img2 = tf.placeholder(tf.float32,[None,150,150,3],'img2')
    img1id = tf.placeholder(tf.int32,[None,],'img1id')
    img2id = tf.placeholder(tf.int32,[None,],'img2id')
    label = tf.placeholder(tf.float32,[None,2],'label')
    learning_rate = tf.placeholder(tf.float32,[],name='learning_rate')
    keep_prob = tf.placeholder(tf.float32,[],name='keep_prob')
    return img1,img2,img1id,img2id,label,learning_rate,keep_prob

In [ ]:
def extractor(img):
    conv1 = keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu',data_format='channels_last',kernel_initializer='glorot_normal')(img)
    conv2 = keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu',data_format='channels_last',kernel_initializer='glorot_normal')(conv1)
    pool1 = keras.layers.MaxPool2D(pool_size=(16,16),padding='same')(conv1)
    pool2 = keras.layers.MaxPool2D(pool_size=(16,16),padding='same')(conv2)
    concat = keras.layers.Concatenate(axis=-1)([pool1,pool2])
    globalpool = keras.layers.GlobalAveragePooling2D()(concat)
    return concat,globalpool

In [ ]:
def image_to_image_attention(conv,globalpool):#conv=[batch_size,14,14,64]，globalpool=[batch_size,64] 
    weights1 = tf.get_variable('weights1',shape=[64,64],initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    weights2 = tf.get_variable('weights2',shape=[64,64],initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    weights3 = tf.get_variable('weights3',shape=[64,1],initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    attn_from = tf.matmul(globalpool,weights1)#attn_form=[batch_size,64]
    features = keras.layers.Reshape([-1,64])(conv)#features=[batch_size,196,64] 
    attn_to = tf.matmul(tf.reshape(features,[-1,64]),weights2)#tf.reshape(features,[-1,64])=[batch_size*196,64]，attn_to=[batch_size*196,64]
    attn_from  = tf.expand_dims(attn_from,1)#attn_from=[batch_size,1,64]
    attn_to = tf.reshape(attn_to,tf.shape(features))#attn_to=[batch_size,196,64] 
    attn_logit = tf.add(attn_from,attn_to)#attn_logit=[batch_size,196,64]
    attn_logit = tf.reshape(attn_logit,[-1,64])#attn_logit=[batch_size*196,64]
    attn_logit = tf.tanh(attn_logit)
    attn_weight = tf.matmul(attn_logit,weights3)#attn_weight=[batch_size*196,1]
    attn_weight = tf.reshape(attn_weight,shape=[tf.shape(conv)[0],tf.shape(conv)[1]*tf.shape(conv)[2]])#attn_weight=[batch_size,196]
    attn_weight = tf.nn.softmax(attn_weight,name='attention_img2img')
    attn_weight = tf.expand_dims(attn_weight,-1)#attn_weight=[batch_size,196,1]
    attn_conv = tf.multiply(features,attn_weight)#attn_conv=[batch_size,196,64]
    attn_conv = tf.reduce_sum(attn_conv,axis=1)#attn_conv=[batch_size,64]
    return features,attn_conv#e=v^Ttanh(W1s+W2h)，a=softmax(e)

In [ ]:
def img2vec(conv):
    extractor_output = keras.layers.Dense(300,activation='relu',kernel_initializer='glorot_normal')(conv)
    return extractor_output

In [ ]:
def img_embedding(img1id,img2id):
    top_embedding_matrix = tf.get_variable('top_embedding_matrix',shape=[len(toplist),embedding_size],initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    down_embedding_matrix = tf.get_variable('down_embedding_matrix',shape=[len(downlist),embedding_size],initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    img1_embedding = tf.nn.embedding_lookup(top_embedding_matrix,img1id)
    img2_embedding = tf.nn.embedding_lookup(down_embedding_matrix,img2id)
    return img1_embedding,img2_embedding

In [ ]:
def classifier(extractor_output,keep_prob):
    dense = keras.layers.Dense(256,activation='relu',kernel_initializer='glorot_normal')(extractor_output)
    dropout = tf.nn.dropout(dense,keep_prob)
    classifier_output = keras.layers.Dense(2,activation='softmax',kernel_initializer='glorot_normal')(dropout)
    return classifier_output

In [ ]:
def loss(classifier_output,label):
    classifier_loss = tf.reduce_mean(tf.contrib.keras.losses.categorical_crossentropy(label,classifier_output),name='classifier_loss')
    tv = tf.trainable_variables()
    reg_loss = tf.reduce_sum([tf.nn.l2_loss(v) for v in tv])
    loss = tf.add(classifier_loss,0.00001*reg_loss,name='loss')
    return loss

In [ ]:
def optimizer(loss,learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss)
    capped_gradients = [(tf.clip_by_value(grad,-5.,5.),var) for grad,var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    return train_op

In [ ]:
def prediction(classifier_output):
    prediction = tf.identity(classifier_output,name='prediction')
    return prediction

In [ ]:
embedding_size = 300
train_graph = tf.Graph()
with train_graph.as_default():
    tf.set_random_seed(1)
    with tf.name_scope('inputs'):
        img1,img2,img1id,img2id,label,learning_rate,keep_prob = get_input()
    with tf.name_scope('extractor'):
        with tf.variable_scope('extractor'):
            conv_img1,globalpool_img1 = extractor(img1)
        with tf.variable_scope('extractor',reuse=True):
            conv_img2,globalpool_img2 = extractor(img2)
        with tf.variable_scope('image_to_image_attention'):
            features_img1,attn_conv_img1 = image_to_image_attention(conv_img1,globalpool_img2)
        with tf.variable_scope('image_to_image_attention',reuse=True):
            features_img2,attn_conv_img2 = image_to_image_attention(conv_img2,globalpool_img1)
        with tf.variable_scope('img2vec'):
            extractor_output_img1 = img2vec(attn_conv_img1)
        with tf.variable_scope('img2vec',reuse=True):
            extractor_output_img2 = img2vec(attn_conv_img2)
        with tf.variable_scope('img_embedding'):
            img1_embedding,img2_embedding = img_embedding(img1id,img2id)
        encoder_output = tf.concat([features_img1,features_img2],axis=1,name='encoder_output')
        extractor_output = tf.concat([extractor_output_img1,extractor_output_img2,img1_embedding,img2_embedding],axis=1,name='extractor_output')
    with tf.name_scope('classifier'):
        classifier_output = classifier(extractor_output,keep_prob)
    with tf.name_scope('prediction'):
        prediction = prediction(classifier_output)
    with tf.name_scope('loss'):
        loss = loss(classifier_output,label)
    with tf.name_scope('optimizer'): 
        train_op = optimizer(loss,learning_rate)   

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Train Model

In [ ]:
with open('/content/drive/MyDrive/Mini_project/CG/train/newtraindata.dat','r') as f:#in traindata, the first col is img_name of top, the second col is img_name of down(i.e. bottom)  
    content = f.readlines()
traindata = []
for line in content:
    line = line[:-1].split('\t')
    traindata.append((line[0],line[1],1))
len(traindata)

199

In [ ]:
type(traindata[1])
#temp = traindata[198]
temp = traindata.pop()
temp
tup = ('179389915', '190945208', 1)
traindata.append(tup)
#traindata

In [ ]:
tops_qrel_file_path = '/content/drive/MyDrive/Mini_project/CG/eval/devdata_tops_qrel.dat'
tops_trec_file_path = '/content/drive/MyDrive/Mini_project/CG/eval/devdata_tops_trec.dat'
#downs_qrel_file_path = 'evaluation/devdata_downs_qrel.dat'
#downs_trec_file_path = 'evaluation/devdata_downs_trec.dat'

In [ ]:
def prepare_evaluation(data_path):  
    with open(data_path,'r') as f:
        content = f.readlines()
    data = {}
    orderlist = []
    labellist = {}
    query_number = 0
    for line in content:
        lines = line.split('\t')
        lines[2]=lines[2][0:1]
        #print(line)
        if data.get(line[0]) != None:
            data[lines[0]].append(lines[1])
        else:
            data[lines[0]] = [lines[1]] 
            labellist[query_number] = {}
            query_number += 1
            orderlist.append(lines[0])
        if int(lines[2]) == 1:
            labellist[query_number-1][lines[1]] = 1
        else:
            labellist[query_number-1][lines[1]] = 0
    return data,orderlist,labellist

In [ ]:
data_path = '/content/drive/MyDrive/Mini_project/CG/train/newdevdata_tops.dat'
dev_tops_data,tops_orderlist,tops_labellist = prepare_evaluation(data_path)
#tops_orderlist

In [ ]:
#data_path = 'dataset/devdata_downs.dat'
#dev_downs_data,downs_orderlist,downs_labellist = prepare_evaluation(data_path)

In [ ]:
lr = 0.001
epochs = 5
rate = 1.0
batch_size = 64

In [ ]:
cost_list = []
auc_tops = []
trec_evals_tops = []
#trec_evals_downs = []
#auc_downs = []

In [ ]:
checkpoint = 'checkpoint/trained_model.ckpt'
with tf.Session(graph=train_graph,config=config) as sess:
    writer = tf.summary.FileWriter('checkpoint/',sess.graph)
    saver = tf.train.Saver(max_to_keep=30)
    sess.run(tf.global_variables_initializer())
    print(time.localtime())
    classifier_loss = train_graph.get_tensor_by_name('loss/classifier_loss:0')
    for epoch in range(epochs):
        train_cost = 0
        temp_cost_list = []
        step = 0
        for _,(x_i1,x_i2,x_id1,x_id2,y_l) in enumerate(get_batches(traindata,batch_size,toplist,downlist,combinationlist,imglist,topidlist,downidlist)):
            _,cost = sess.run([train_op,classifier_loss],{img1:x_i1,img2:x_i2,img1id:x_id1,img2id:x_id2,label:y_l,learning_rate:lr,keep_prob:rate})
            train_cost += cost
            step += 1
            if step%1000 == 0:
                temp_cost_list.append(train_cost/step)
                print(str(train_cost/step)+' '+'pass!')
        temp_cost_list.append(train_cost/step)
        cost_list.append(temp_cost_list)
        print('Epoch {}/{} - Training Loss: {:.3f}'.format(epoch+1,epochs,train_cost/step))
        saver.save(sess,checkpoint,global_step=epoch+1)
        print('Model Trained and Saved')
        print(time.localtime())
        #validation      
        tops_trec = {}
        query_number = 0
        step = 0
        for top in tops_orderlist:
            downsoftop = dev_tops_data[top]
            probabilitylist = {}
            for batch_i in range(len(downsoftop)//batch_size+1):
                start_i = batch_i*batch_size
                downs = downsoftop[start_i:start_i+batch_size]
                x_i1,x_i2,x_id1,x_id2 = build_evaluation_batch(top,downs,0,imglist,topidlist,downidlist)
                seq_len = [30]*len(x_i1)
                prob = sess.run(prediction,{img1:x_i1,img2:x_i2,img1id:x_id1,img2id:x_id2,keep_prob:1.0})
                j = 0
                for down in downs:
                    probabilitylist[down] = prob[j][1]
                    j += 1 
                step += 1
                if step%1000 == 0:
                    print('pass!')
            tops_trec[query_number] = sorted(probabilitylist.items(),key=lambda item:item[1],reverse=True)
            del probabilitylist,downsoftop
            query_number += 1
        auc_tops.append(auc_evaluation(tops_labellist,tops_trec))
        trec_evals_tops.append(trec_evaluation(tops_qrel_file_path,tops_trec_file_path,tops_trec))
        del tops_trec
        '''
        downs_trec = {}
        query_number = 0
        step = 0
        for down in downs_orderlist:
            topsofdown = dev_downs_data[down]
            probabilitylist = {}
            for batch_i in range(len(topsofdown)//batch_size+1):
                start_i = batch_i*batch_size
                tops = topsofdown[start_i:start_i+batch_size]
                x_i1,x_i2,x_id1,x_id2 = build_evaluation_batch(down,tops,1,imglist,topidlist,downidlist)
                prob = sess.run(prediction,{img1:x_i1,img2:x_i2,img1id:x_id1,img2id:x_id2,keep_prob:1.0})
                j = 0
                for top in tops:
                    probabilitylist[top] = prob[j][1]
                    j += 1
                step += 1
                if step%1000 == 0:
                    print('pass!')
            downs_trec[query_number] = sorted(probabilitylist.items(),key=lambda item:item[1],reverse=True)
            del probabilitylist,topsofdown
            query_number += 1
        auc_downs.append(auc_evaluation(downs_labellist,downs_trec))
        trec_evals_downs.append(trec_evaluation(downs_qrel_file_path,downs_trec_file_path,downs_trec))
        del downs_trec
        '''
        #validation      
        print(time.localtime())

time.struct_time(tm_year=2022, tm_mon=6, tm_mday=25, tm_hour=17, tm_min=37, tm_sec=47, tm_wday=5, tm_yday=176, tm_isdst=0)
Epoch 1/5 - Training Loss: 0.717
Model Trained and Saved
time.struct_time(tm_year=2022, tm_mon=6, tm_mday=25, tm_hour=17, tm_min=38, tm_sec=37, tm_wday=5, tm_yday=176, tm_isdst=0)
161425425
183639308
149494468
185256499
187619920
192304171
154908928
195127827
201020544
184706162
150894162
181131950
179103801
189834308
119380232
188007967
188977857
182883012
190945208
190044021
195070550
184478646
187329463
164683144
181298080
144475071
195590627
191300349
187264196
179734357
134944806
185315961
154789765
182028192
194791452
95459757
154937089
149682245
192966244
196453490
161361175
123524655
194004637
166367125
114499573
196049545
174451466
166460249
168939642
183486674
time.struct_time(tm_year=2022, tm_mon=6, tm_mday=25, tm_hour=17, tm_min=38, tm_sec=40, tm_wday=5, tm_yday=176, tm_isdst=0)
Epoch 2/5 - Training Loss: 0.692
Model Trained and Saved
time.struct_time(t

# Evaluate Model

In [ ]:
tops_qrel_file_path = '/content/drive/MyDrive/Mini_project/CG/eval/testdata_tops_qrel.dat'
tops_trec_file_path = '/content/drive/MyDrive/Mini_project/CG/eval/testdata_tops_trec.dat'
downs_qrel_file_path = '/content/drive/MyDrive/Mini_project/CG/eval/testdata_downs_qrel.dat'
downs_trec_file_path = '/content/drive/MyDrive/Mini_project/CG/eval/testdata_downs_trec.dat'

In [ ]:
data_path = '/content/drive/MyDrive/Mini_project/CG/train/newtestdata_tops.dat'
test_tops_data,tops_orderlist,tops_labellist = prepare_evaluation(data_path)

In [ ]:
data_path = '/content/drive/MyDrive/Mini_project/CG/train/newtestdata_downs.dat'
test_downs_data,downs_orderlist,downs_labellist = prepare_evaluation(data_path)

In [ ]:
print(time.localtime())
checkpoint = 'checkpoint/trained_model.ckpt-2'
test_graph = tf.Graph()
with tf.Session(graph=test_graph,config=config) as sess:
    loader = tf.train.import_meta_graph(checkpoint+'.meta')
    loader.restore(sess,checkpoint)
    img1 = test_graph.get_tensor_by_name('inputs/img1:0')
    img2 = test_graph.get_tensor_by_name('inputs/img2:0')
    img1id = test_graph.get_tensor_by_name('inputs/img1id:0')
    img2id = test_graph.get_tensor_by_name('inputs/img2id:0')
    keep_prob = test_graph.get_tensor_by_name('inputs/keep_prob:0')
    prediction = test_graph.get_tensor_by_name('prediction/prediction:0')
    batch_size = 64
    tops_trec = {}
    query_number = 0
    step = 0
    for top in tops_orderlist:
        downsoftop = test_tops_data[top]
        probabilitylist = {}
        for batch_i in range(len(downsoftop)//batch_size+1):
            start_i = batch_i*batch_size
            downs = downsoftop[start_i:start_i+batch_size]
            x_i1,x_i2,x_id1,x_id2 = build_evaluation_batch(top,downs,0,imglist,topidlist,downidlist)
            prob = sess.run(prediction,{img1:x_i1,img2:x_i2,img1id:x_id1,img2id:x_id2,keep_prob:1.0})
            j = 0
            for down in downs:
                probabilitylist[down] = prob[j][1]
                j += 1 
            step += 1
            if step%1000 == 0:
                print('pass!')
        tops_trec[query_number] = sorted(probabilitylist.items(),key=lambda item:item[1],reverse=True)
        del probabilitylist,downsoftop
        query_number += 1
    auc_evaluation(tops_labellist,tops_trec)
    trec_evaluation(tops_qrel_file_path,tops_trec_file_path,tops_trec)
    del tops_trec
    downs_trec = {}
    query_number = 0
    step = 0
    for down in downs_orderlist:
        topsofdown = test_downs_data[down]
        probabilitylist = {}
        for batch_i in range(len(topsofdown)//batch_size+1):
            start_i = batch_i*batch_size
            tops = topsofdown[start_i:start_i+batch_size]
            x_i1,x_i2,x_id1,x_id2 = build_evaluation_batch(down,tops,1,imglist,topidlist,downidlist)
            prob = sess.run(prediction,{img1:x_i1,img2:x_i2,img1id:x_id1,img2id:x_id2,keep_prob:1.0})
            j = 0
            for top in tops:
                probabilitylist[top] = prob[j][1]
                j += 1
            step += 1
            if step%1000 == 0:
                print('pass!')
        downs_trec[query_number] = sorted(probabilitylist.items(),key=lambda item:item[1],reverse=True)
        del probabilitylist,topsofdown
        query_number += 1
    auc_evaluation(downs_labellist,downs_trec)
    trec_evaluation(downs_qrel_file_path,downs_trec_file_path,downs_trec)
    del downs_trec
print(time.localtime())

time.struct_time(tm_year=2022, tm_mon=6, tm_mday=25, tm_hour=17, tm_min=44, tm_sec=10, tm_wday=5, tm_yday=176, tm_isdst=0)
INFO:tensorflow:Restoring parameters from checkpoint/trained_model.ckpt-2
143550669
200579227
83552962
197269090
106239071
184587206
194152283
179374468
196352343
167541452
189696485
172528623
136462851
163736626
140964599
168779328
162873346
185310648
113657786
176961116
191012491
160706489
170992137
199596946
193124361
163750225
157087241
159128708
196007836
191612229
117016854
191597987
188030741
183935563
101347443
177292461
166189276
177896547
153637817
163833207
188036380
192219758
170987568
114600316
161136849
196487907
182218888
197738898
186323478
171249641
time.struct_time(tm_year=2022, tm_mon=6, tm_mday=25, tm_hour=17, tm_min=44, tm_sec=14, tm_wday=5, tm_yday=176, tm_isdst=0)
